# CHIRPS V3 Data Extractor

Extract CHIRPS V3.0 precipitation data for specific coordinates and date ranges.

In [2]:
# Configuration Constants
LATITUDE = 42.0
LONGITUDE = -93.5
START_DATE = '2020-01-01'
END_DATE = '2020-01-31'
DATA_DIR = './chirps_v3_data'
CHIRPS_V3_BASE_URL = 'https://data.chc.ucsb.edu/products/CHIRPS/v3.0/daily/final/rnl'

In [3]:
import os
import requests
import pandas as pd
import rioxarray
from datetime import datetime
from calendar import monthrange
from pathlib import Path

In [4]:
def check_existing_files(start_date, end_date, data_dir):
    """Check which CHIRPS V3 files already exist locally."""
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    Path(data_dir).mkdir(parents=True, exist_ok=True)
    
    existing_files = []
    missing_dates = []
    
    current = start
    while current <= end:
        year = current.year
        month = current.month
        day = current.day
        
        filename = f'chirps-v3.0.rnl.{year}.{month:02d}.{day:02d}.tif'
        filepath = Path(data_dir) / filename
        
        if filepath.exists():
            existing_files.append(filepath)
        else:
            missing_dates.append(current)
        
        current += pd.Timedelta(days=1)
    
    total_days = (end - start).days + 1
    return {
        'existing': existing_files,
        'missing': missing_dates,
        'total': total_days,
        'existing_count': len(existing_files),
        'missing_count': len(missing_dates)
    }

In [5]:
def download_chirps_v3(missing_dates, data_dir, base_url):
    """Download missing CHIRPS V3 TIF files."""
    downloaded = []
    failed = []
    
    for date in missing_dates:
        year = date.year
        month = date.month
        day = date.day
        
        filename = f'chirps-v3.0.rnl.{year}.{month:02d}.{day:02d}.tif'
        filepath = Path(data_dir) / filename
        url = f'{base_url}/{year}/{filename}'
        
        try:
            response = requests.get(url, stream=True, timeout=300)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            downloaded.append(filepath)
            
        except Exception as e:
            failed.append((filename, str(e)))
    
    return {'downloaded': downloaded, 'failed': failed}

In [6]:
def load_chirps_data(file_paths, lat, lon):
    """Load CHIRPS V3 data and extract values for specific coordinates."""
    data = []
    
    for filepath in sorted(file_paths):
        filename = filepath.name
        parts = filename.split('.')
        year = int(parts[3])
        month = int(parts[4])
        day = int(parts[5])
        
        da = rioxarray.open_rasterio(filepath, masked=True)
        da = da.squeeze().drop_vars('band', errors='ignore')
        
        point_value = da.sel(x=lon, y=lat, method='nearest').values
        
        if hasattr(point_value, 'item'):
            point_value = point_value.item()
        
        if point_value < 0 or pd.isna(point_value):
            point_value = 0.0
        
        data.append({
            'year': year,
            'month': month,
            'day': day,
            'precip': float(point_value)
        })
        
        da.close()
    
    return data

In [ ]:
def create_dataframe(data):
    """Convert data to DataFrame with DATE (yyyyddd) and CRAIN columns."""
    df = pd.DataFrame(data)
    
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['day_of_year'] = df['date'].dt.dayofyear
    df['DATE'] = df['year'] * 1000 + df['day_of_year']
    df['RAIN'] = df['precip']
    
    result = df[['DATE', 'RAIN']].copy()
    
    return result

## Execution

In [17]:
file_status = check_existing_files(START_DATE, END_DATE, DATA_DIR)
file_status

{'existing': [WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.01.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.02.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.03.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.04.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.05.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.06.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.07.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.08.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.09.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.10.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.11.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.12.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.13.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.14.tif'),
  WindowsPath('chirps_v3_data/chirps-v3.0.rnl.2020.01.15.tif'),
  WindowsPath('chirps_v3_dat

In [19]:
if file_status['missing_count'] > 0:
    download_result = download_chirps_v3(file_status['missing'], DATA_DIR, CHIRPS_V3_BASE_URL)
    download_result
else:
    print(f"All {file_status['total']} files already exist. Skipping download.")

All 31 files already exist. Skipping download.


In [10]:
all_files = check_existing_files(START_DATE, END_DATE, DATA_DIR)['existing']
raw_data = load_chirps_data(all_files, LATITUDE, LONGITUDE)
raw_data[:5]

[{'year': 2020, 'month': 1, 'day': 1, 'precip': 0.0},
 {'year': 2020, 'month': 1, 'day': 2, 'precip': 0.0},
 {'year': 2020, 'month': 1, 'day': 3, 'precip': 0.0},
 {'year': 2020, 'month': 1, 'day': 4, 'precip': 0.0},
 {'year': 2020, 'month': 1, 'day': 5, 'precip': 0.0}]

In [11]:
chirps_df = create_dataframe(raw_data)
chirps_df

,DATE,CRAIN
0,2020001,0.000000
1,2020002,0.000000
2,2020003,0.000000
3,2020004,0.000000
4,2020005,0.000000
5,2020006,0.014955
6,2020007,0.079714
7,2020008,0.000000
8,2020009,0.263693
9,2020010,3.433313


## Summary

The notebook successfully:
1. **Checks** for existing CHIRPS V3 files (skips already downloaded)
2. **Downloads** missing daily TIF files from CHC Portal
3. **Extracts** precipitation values for specified coordinates
4. **Returns** DataFrame with:
   - `DATE`: 7-digit format (yyyyddd) - e.g., 2020001 = Jan 1, 2020
   - `CRAIN`: Precipitation in mm